In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [189]:
df = pd.read_csv('iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [190]:
df.head()
df['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [191]:
df['Species']=df['Species'].map({"Iris-setosa":0,"Iris-versicolor":1,"Iris-virginica":2})
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,2
146,147,6.3,2.5,5.0,1.9,2
147,148,6.5,3.0,5.2,2.0,2
148,149,6.2,3.4,5.4,2.3,2


In [192]:
def entropy(y):
    unique,counts = np.unique(y,return_counts=True)

    t=np.sum(counts)
    probabilities =counts/counts.sum()

    return -np.sum(probabilities*np.log2(probabilities+1e-9))

In [ ]:
import numpy as np
import pandas as pd

def entropy(y):
    unique,counts =np.unique(y,return_counts=True)
    prob = counts/sum(counts)
    return -np.sum(prob*np.log2(prob+1e-9))

def information_gain(x,y,threshold):
    n=len(y)
    xleft_ind = x<=threshold
    xright_ind= x>threshold
    if (sum(xleft_ind)==0 or sum(xright_ind)==0):
        return 0
    leftentropy= entropy(y[xleft_ind])
    rightentropy= entropy(y[xright_ind])

    nl,nr = sum(xleft_ind),sum(xright_ind)

    weighted_entropy =(nl/n)*leftentropy+(nr/n)*rightentropy

    return entropy(y)-weighted_entropy

class Node:
    def __init__(self,feature=None,threshold=None,left=None,right=None,value=None):
        self.feature =feature
        self.threshold =threshold
        self.left =left
        self.right =right
        self.value =value

class DT:
    def __init__(self,max_depth=None):
        self.max_depth =max_depth
        self.root =None
    
    def fit(self,x,y):
        self.root=self.grow_tree(x,y)
    
    def most_common_label(self,y):
        unique,counts =np.unique(y,return_counts= True)
        return unique[np.argmax(counts)]
    
    def grow_tree(self,x,y,depth=0):

        n_samples,n_features =x.shape
        n_labels= len(np.unique(y))

        if (depth>= self.max_depth or n_labels ==1 or n_samples==0):
            leaf_value = self.most_common_label(y)
            return Node(value=leaf_value)
        
        best_gain=-1
        split_feature,split_thresh =None,None
        for feature_id in range(n_features):
            x_column= x.iloc[:,feature_id]
            thresholds =np.unique(x_column)

            for threshold in thresholds:
                gain =information_gain(x_column,y,threshold)
                if gain>best_gain:
                    best_gain=gain
                    split_thresh=threshold
                    split_feature=feature_id
        
        left_indices = x.iloc[:,split_feature]<=split_thresh
        right_indices = x.iloc[:,split_feature]>split_thresh

        left= self.grow_tree(x[left_indices],y[left_indices],depth+1)
        right =self.grow_tree(x[right_indices],y[right_indices],depth+1)

        return Node(split_feature,split_thresh,left,right)
    
    def traverse_tree(self,x,node):
        if node.value is not None:
            return node.value
        if x[node.feature]<= node.threshold:
            return self.traverse_tree(x,node.left)
        return self.traverse_tree(x,node.right)
    
    def predict(self,x):
        return np.array([self.traverse_tree(x,self.root)])

In [194]:
from sklearn.model_selection import train_test_split

x=df.iloc[:,[1,2,3,4]]
y=df.iloc[:,5]
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=42)

In [195]:
dt= DT(3)
dt.fit(x_train,y_train)
mismatches =0
for i in range(len(x_test)):
    prediction = dt.predict(x_test.iloc[i,:])
    # print(prediction[0])
    mismatches += abs(prediction[0]-y_test.iloc[i])
    # print(mismatches)
    # print("predictions \n",prediction)

print(mismatches/len(x_test))


0.0


C:\Users\venka\AppData\Local\Temp\ipykernel_23576\2285376646.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[node.feature]<= node.threshold:
